# Signifikanstestning

Innan vi ger en rekommendation så bör vi kontrollera att det rör sig om en statistiskt säkerställd skillnad och inte bara en slumpmässig variation.

Här tittar vi på hur man kan göra detta från en Jupyter notebook. Det första vi behöver göra är att importera ett antal Python-bibliotek som låter oss läsa in data från Google Analytics, samt enklare kunna hantera vår data. 


In [47]:
import sys
!{sys.executable} -m pip install requests --user

import pandas as pd 
import json
import requests

/home/ros/.local/lib/python2.7/site-packages/pip/_vendor/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
You should consider upgrading via the 'pip install --upgrade pip' command.


## Fishers exakta test

Har man väldigt lite data kan man använda sig av Fishers exakta test. 

Låt oss använda detta för att kolla om det är någon statistisk skillnad i bounce rate mellan besökare som kommer från sociala media och användare som kommer via referral.

I det här fallet är vi intresserade av att läsa in totalt antal sessions och totalt antal bounces för
Vi använder oss av Google Analytics Query Explorer för att välja ut den data vi vill läsa in:

https://ga-dev-tools.appspot.com/query-explorer/?ids=ga%3A130903410&start-date=30daysAgo&end-date=yesterday&metrics=ga%3Asessions%2Cga%3Abounces&dimensions=ga%3AchannelGrouping

Query explorer låter oss enkelt välja ut den data vi vill ha och längst ner på sidan hittar vi en länk till det API som används för att läsa ut data. Vi kan kopiera den länken och sedan göra ett anrop till denna med Python för att läsa in datan i vår notebook.



In [58]:


r = requests.get('https://www.googleapis.com/analytics/v3/data/ga?ids=ga%3A130903410&start-date=30daysAgo&end-date=yesterday&metrics=ga%3Asessions%2Cga%3Abounces&dimensions=ga%3AchannelGrouping&access_token=ya29.Il-vB6a4yiK_LRwQpZDdOjJqPDgvmAdkhqPW9AD3LjOkzFGyYNqfdMWArsnRimP1o56ZhFDRn2ysiphKaNzvJQ6dp9Y2k6dnpEUOfb9JocYi4PEE784R0oD2MRcu3-tJ4Q')

data = r.json() # read data from a JSON format
df = pd.DataFrame(data['rows']) # turn data into a Pandas data frame
df = df.rename(columns={0: 'Channel', 1: 'Sessions', 2: 'Bounces'}) # giving the columns some proper titles
df['Sessions'] = df['Sessions'].astype(int) # formatting sessions as ints
df['Bounces'] = df['Bounces'].astype(int) # formatting users as ints

df # printing the first five rows


,Channel,Sessions,Bounces
0,(Other),137,39
1,Direct,315,94
2,Display,5,3
3,Email,146,62
4,Organic Search,146,60
5,Paid Search,2,1
6,Referral,466,169
7,Social,193,104


In [61]:

social = df[df.Channel == 'Social']
social_bounces = social['Bounces'].values[0]
social_non_bounces = social['Sessions'].values[0] - social['Bounces'].values[0]

print (social_bounces)
print (social_non_bounces) 

referral = df[df.Channel == 'Referral']
referral_bounces = referral['Bounces'].values[0]
referral_non_bounces = referral['Sessions'].values[0] - referral['Bounces'].values[0]

print (referral_bounces)
print (referral_non_bounces) 


104
89
169
297


In [62]:
import scipy.stats as stats

oddsratio, pvalue = stats.fisher_exact([[social_bounces, referral_bounces], [social_non_bounces, referral_non_bounces]])

print (pvalue)

4.1473086347635335e-05
